# Day 5 Logistic Regression

logistic regression은 binary classification 문제이다.<br>
예를 들어 d size의 벡터에 m개의 데이터가 들어있다고 한다면, 이 행렬을 m개의 0과 1로 이루어진 정답을 구하는 모델을 학습해야 할 것이다. 이것이 binary calssification의 세팅이 될 것이다.

$$ X \in \mathbb{R}^{m * d} $$

어떤 확률 P가 1일 확률은 1에서 어떤 확률 P가 0일 확률을 뺀 값일 것이다.

$$ P(X = 1) = 1 - P(X = 0) $$

Logistic Regression의 Weight parameter는 보통 W로 불리우는데,<br>
W는 (d,1) 개의 weight parameter가 들어있다.

$$ W \in \mathbb{R}^{d * 1} $$

X와 W를 곱한 후, sigmoid 함수를 사용하여 0과 1에 근사하는 값을 도출한다.

$$ Sigmoid 함수 = 

-\infty 는 
0,
+\infty 는 1 $$
의 값을 갖게 해주는 함수이다:

$$ \sigma = \frac{1}{1+e^{-x}} $$

이를 차용한 것이 H(X)이다.<br>
H(X)는 P(X=1), 즉 X가 1일 확률을 의미한다.


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
# For reproducibility
torch.manual_seed(1)

## Training Data

In [24]:
x_data = [[1,2],
          [2,3],
          [3,1],
          [4,3],
          [5,3],
          [6,2]]
y_data = [[0],
          [0],
          [0],
          [1],
          [1],
          [1]]

In [43]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
x_test = torch.FloatTensor(x_data)

In [26]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


Pytorch has a `torch.exp()` function that resembles the exponential function

In [11]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


We can use it to compute the hypothesis function conveniently.

In [17]:
W = torch.zeros((2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

print('W: ', W)
print('b: ', b)

W:  tensor([[0.],
        [0.]], requires_grad=True)
b:  tensor([0.], requires_grad=True)


In [19]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W)+b)))

# 또는 이렇게도 표현할 수 있다.
hypothesis2 = 1 / (1+ torch.exp(-torch.matmul(x_train,W)+b))

In [20]:
print(x_train.matmul(W))

print(hypothesis)
print(hypothesis.shape)

print(hypothesis2)

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<MmBackward0>)
tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])
tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)


Pytorch has a `torch.sigmoid()` function 

In [27]:
print('1/(1+e^{-1}): ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}):  tensor([0.7311])


In [29]:
hypothesis = torch.sigmoid(-(x_train.matmul(W)+b))


print(hypothesis)
print(hypothesis.shape)


tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


We want to measure the difference between `hypothesis` and `y_train`

In [31]:
print('hypothesis: \n',hypothesis)
print('\n y_train: \n',y_train)

hypothesis: 
 tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)

 y_train: 
 tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


## Computing the Cost Function
For the element, the loss can be computed as follows:

In [33]:
-(y_train[0] * torch.log(hypothesis[0]) +
  (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward0>)

To compute the losses for the entire batch, we can simply input the entire vector

In [35]:
losses = -(y_train * torch.log(hypothesis[0]) +
  (1 - y_train) * torch.log(1 - hypothesis[0]))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


Then, we just `.mean()` to take the mean of these individual lossess

In [36]:
cost = losses.mean()
print('cost: \n', cost)

cost: 
 tensor(0.6931, grad_fn=<MeanBackward0>)


In [37]:
# 이 한 줄이면 간단하게 같은 값을 구할 수 있다.
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

## Whole Training Procedure

In [42]:
# Model initialization
W = torch.zeros((2,1), requires_grad=True)
b = torch.zeros((1), requires_grad=True)

# Optimizer
optimizer = optim.SGD([W,b], lr = 1)

# Epochs
nb_epochs = 1000

# Training
for epoch in range(nb_epochs+1):

    # Cost
    hypothesis = torch.sigmoid(x_train.matmul(W)+b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # H(X)
    optimizer.zero_grad() # 혹시 이전에 계산한 Gradient 가 있을 수 있으니 0으로 변경
    cost.backward()
    optimizer.step()

    # Print log in every 100 epoch
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


## Evalution
After we finish training the model, we want to check how well our model fits the training set.

In [45]:
import numpy as np

In [50]:
xy = np.loadtxt('/Volumes/SUHYE/1.research/2024_study/PyTorch/data-03-diabetes.csv'
                , delimiter=',', dtype= np.float32)
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# print('x_data: \n', x_data) 
# print('y_data: \n', y_data)
# print('x_train: \n', x_train)
# print('y_train: \n', y_train)


In [54]:
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/100 Cost: 0.693147
Epoch   10/100 Cost: 0.572727
Epoch   20/100 Cost: 0.539493
Epoch   30/100 Cost: 0.519708
Epoch   40/100 Cost: 0.507066
Epoch   50/100 Cost: 0.498539
Epoch   60/100 Cost: 0.492549
Epoch   70/100 Cost: 0.488209
Epoch   80/100 Cost: 0.484985
Epoch   90/100 Cost: 0.482543
Epoch  100/100 Cost: 0.480661


In [55]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/100 Cost: 0.693147
Epoch   10/100 Cost: 0.572727
Epoch   20/100 Cost: 0.539493
Epoch   30/100 Cost: 0.519708
Epoch   40/100 Cost: 0.507066
Epoch   50/100 Cost: 0.498539
Epoch   60/100 Cost: 0.492549
Epoch   70/100 Cost: 0.488209
Epoch   80/100 Cost: 0.484985
Epoch   90/100 Cost: 0.482543
Epoch  100/100 Cost: 0.480661


In [56]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[0.4103],
        [0.9242],
        [0.2300],
        [0.9411],
        [0.1772]], grad_fn=<SliceBackward0>)


In [63]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5].float())

tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


Then, we compare it with the correcdt labels `y_train`.

In [66]:
print('prediction: \n',prediction[:5].float())
print('\ny_train: \n', y_train[:5])


prediction: 
 tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])

y_train: 
 tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


In [70]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5].float())

accuracy = torch.mean(correct_prediction.float())
print('\nACC: \n', accuracy)

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]])

ACC: 
 tensor(0.7668)


## Higher Implementation with class

In [72]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8,1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [78]:
model = BinaryClassifier()

## Higher Implemenation with Class

In [82]:
# Optimizer
optimizer = optim.SGD(model.parameters(), lr = 0.01)

nb_epochs = 100

for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train) # x가 1일 확률 P(x=1)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # print log in every 10 times
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)

        print('Epoch {:4d}/{} Cost: {:.6f}, Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy*100
        ))
        


Epoch    0/100 Cost: 0.474036, Accuracy 76.94%
Epoch   10/100 Cost: 0.474033, Accuracy 76.94%
Epoch   20/100 Cost: 0.474031, Accuracy 76.94%
Epoch   30/100 Cost: 0.474028, Accuracy 76.94%
Epoch   40/100 Cost: 0.474026, Accuracy 76.94%
Epoch   50/100 Cost: 0.474023, Accuracy 76.94%
Epoch   60/100 Cost: 0.474021, Accuracy 76.94%
Epoch   70/100 Cost: 0.474018, Accuracy 76.94%
Epoch   80/100 Cost: 0.474016, Accuracy 76.94%
Epoch   90/100 Cost: 0.474014, Accuracy 76.94%
Epoch  100/100 Cost: 0.474011, Accuracy 76.94%


Latex 기호 모음
https://jjycjnmath.tistory.com/117